# Test the endpoint

In [84]:
import json
import boto3
import sagemaker
import pandas as pd
import numpy as np

client = boto3.client('sagemaker-runtime')

In [85]:
input_txt = "test my doctor has "

In [86]:
df = pd.read_csv("data/mtsamples.csv")

In [88]:
df.transcription[100]

"PREOPERATIVE DIAGNOSIS: , Right inguinal hernia. ,POSTOPERATIVE DIAGNOSIS:,  Right direct inguinal hernia. ,PROCEDURE:,  Right direct inguinal hernia repair with PHS mesh system. ,ANESTHESIA:,  General with endotracheal intubation. ,PROCEDURE IN DETAIL: , The patient was taken to the operating room and placed supine on the operating table.  General anesthesia was administered with endotracheal intubation.  The Right groin and abdomen were prepped and draped in the standard sterile surgical fashion.  An incision was made approximately 1 fingerbreadth above the pubic tubercle and in a skin crease.  Dissection was taken down through the skin and subcutaneous tissue.  Scarpa's fascia was divided, and the external ring was located.  The external oblique was divided from the external ring up towards the anterior superior iliac spine.  The cord structures were then encircled.  Careful inspection of the cord structures did not reveal any indirect sac along the cord structures.  I did, however

In [14]:
# df.medical_specialty.value_counts()
# df[100:120]

## Run Inference on deployed endpoint

In [92]:
idx = 2
input_txt = df.loc[idx].transcription
category = df.loc[idx].medical_specialty
print(category)
print(input_txt)

 Bariatrics
HISTORY OF PRESENT ILLNESS: , I have seen ABC today.  He is a very pleasant gentleman who is 42 years old, 344 pounds.  He is 5'9".  He has a BMI of 51.  He has been overweight for ten years since the age of 33, at his highest he was 358 pounds, at his lowest 260.  He is pursuing surgical attempts of weight loss to feel good, get healthy, and begin to exercise again.  He wants to be able to exercise and play volleyball.  Physically, he is sluggish.  He gets tired quickly.  He does not go out often.  When he loses weight he always regains it and he gains back more than he lost.  His biggest weight loss is 25 pounds and it was three months before he gained it back.  He did six months of not drinking alcohol and not taking in many calories.  He has been on multiple commercial weight loss programs including Slim Fast for one month one year ago and Atkin's Diet for one month two years ago.,PAST MEDICAL HISTORY: , He has difficulty climbing stairs, difficulty with airline seats, 

In [93]:
endpoint_name = "28-2023-04-04-06-38-09-524"
content_type = "text/csv" # "application/json" #
accept = "application/json" #"text/csv"
# payload = "test"#json.dumps({"instances":["test"]})

response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    # CustomAttributes=custom_attributes,
    ContentType=content_type,
    Accept=accept,
    Body=input_txt,
)

json.loads(response["Body"].read())


ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from primary with message "{
  "code": 400,
  "type": "InternalServerException",
  "message": "The size of tensor a (1099) must match the size of tensor b (512) at non-singleton dimension 1"
}
". See https://eu-west-3.console.aws.amazon.com/cloudwatch/home?region=eu-west-3#logEventViewer:group=/aws/sagemaker/Endpoints/28-2023-04-04-06-38-09-524 in account 101436505502 for more information.

## Run batch inference (Transformer)

In [82]:
session.boto_region_name

'eu-west-3'

In [83]:
session = sagemaker.Session()
default_bucket = session.default_bucket()

model_name = "22-2023-04-03-13-43-05-767"
input_data_path = f"s3://{default_bucket}/data/test.csv"
output_data_path = f"s3://{default_bucket}/data/out"
CONTENT_TYPE_CSV = 'text/csv'

iam = boto3.client('iam')
role_arn = iam.get_role(RoleName=f'101436505502-sagemaker-exec')['Role']['Arn']
model_package_arn = "arn:aws:sagemaker:eu-west-3:101436505502:" \
                        f"model-package/training-pipelineModelGroup/22"


# c = boto3.client('sagemaker')
# c.list_model_packages(ModelPackageGroupName="training-pipelineModelGroup")


transform_job = sagemaker.transformer.Transformer(
    model_name = model_name,
    instance_count = 1,
    instance_type = 'ml.g4dn.xlarge', # ml.g4dn.xlarge
    strategy = 'SingleRecord',
    assemble_with = 'Line',
    output_path = output_data_path,
    base_transform_job_name='inference-pipelines-batch',
    sagemaker_session=session,
    accept = CONTENT_TYPE_CSV)

transform_job.transform(data = input_data_path, 
                        content_type = CONTENT_TYPE_CSV, 
                        split_type = 'Line')

INFO:sagemaker:Creating transform job with name: inference-pipelines-batch-2023-04-03-14-12-21-062


ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTransformJob operation: The account-level service limit 'ml.g4dn.xlarge for transform job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 1 Instances. Please contact AWS support to request an increase for this limit.

In [79]:
transform_job.

In [75]:
model = sagemaker.ModelPackage(role=role_arn, model_package_arn=model_package_arn,
                         sagemaker_session=session)

transformer =model.transformer(
    instance_count = 1,
    instance_type = 'ml.g4dn.xlarge', # ml.g4dn.xlarge
    strategy = 'SingleRecord',
    assemble_with = 'Line',
    output_path = output_data_path,
    accept = CONTENT_TYPE_CSV,
    # region="eu-west-3"
)

INFO:sagemaker:Creating model with name: 22-2023-04-03-13-51-02-471


In [76]:
transformer.transform(data = input_data_path, 
                        content_type = CONTENT_TYPE_CSV, 
                        split_type = 'Line')

INFO:sagemaker:Creating transform job with name: 22-2023-04-03-13-51-08-821


ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTransformJob operation: The account-level service limit 'ml.g4dn.xlarge for transform job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 1 Instances. Please contact AWS support to request an increase for this limit.